In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2672/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [7]:
analize = Analizer(0.9)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_11_9_14,0.999488,0.997290,0.996320,0.997243,0.150889,0.343860,0.488276,0.411821,"Hidden Size=[35], regularizer=0.01, learning_r..."
1,model_11_9_13,0.999487,0.997452,0.996457,0.997373,0.151076,0.323351,0.470157,0.392436,"Hidden Size=[35], regularizer=0.01, learning_r..."
2,model_11_9_15,0.999486,0.997139,0.996195,0.997123,0.151431,0.363034,0.504920,0.429805,"Hidden Size=[35], regularizer=0.01, learning_r..."
3,model_11_9_12,0.999483,0.997623,0.996605,0.997512,0.152261,0.301548,0.450492,0.371640,"Hidden Size=[35], regularizer=0.01, learning_r..."
4,model_11_9_16,0.999482,0.996998,0.996080,0.997011,0.152491,0.380861,0.520166,0.446416,"Hidden Size=[35], regularizer=0.01, learning_r..."
5,model_11_9_17,0.999477,0.996868,0.995975,0.996909,0.153912,0.397379,0.534111,0.461724,"Hidden Size=[35], regularizer=0.01, learning_r..."
6,model_11_9_11,0.999474,0.997805,0.996765,0.997661,0.154782,0.278536,0.429225,0.349448,"Hidden Size=[35], regularizer=0.01, learning_r..."
7,model_11_9_18,0.999472,0.996748,0.995879,0.996815,0.155565,0.412616,0.546828,0.475775,"Hidden Size=[35], regularizer=0.01, learning_r..."
8,model_11_9_19,0.999466,0.996638,0.995792,0.996729,0.157359,0.426630,0.558425,0.488651,"Hidden Size=[35], regularizer=0.01, learning_r..."
9,model_11_9_10,0.999460,0.997994,0.996938,0.997818,0.159075,0.254478,0.406324,0.325935,"Hidden Size=[35], regularizer=0.01, learning_r..."
